In [ ]:
# Config



In [ ]:
# Data frames - tables

# Config script
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
#imports

from pyspark.sql.functions import *
import pyspark.sql.functions as sqlfunc


In [ ]:

# data cleaning
data = spark.read.format("csv").option("header", "true").load("anorexia.csv")
data = data.withColumn("Pre_Weight_kg", round(data.Prewt * 0.45359237, 2)).withColumn("Post_Weight_kg", round(data.Postwt * 0.45359237, 2))
data = data.withColumn("Treat", when(data.Treat == "Cont", "Control")
                               .when(data.Treat == "FT", "Family Treatment")
                               .otherwise("CTB"))

data = data.withColumn("Difference", round(data.Post_Weight_kg - data.Pre_Weight_kg, 2))
summary_by_Treat = data.groupby("Treat").agg(sqlfunc.avg('Difference').alias("Average_diff"),
                                             sqlfunc.min("Difference").alias("Minimum diff"),
                                             sqlfunc.max("Difference").alias("Maximum diff")
                                    
                                        )

summary_by_Treat.show()

+----------------+--------------------+------------+------------+
|           Treat|        Average_diff|Minimum diff|Maximum diff|
+----------------+--------------------+------------+------------+
|             CTB|  1.3648275862068968|       -4.13|        9.48|
|         Control|-0.20461538461538456|       -5.54|        7.22|
|Family Treatment|  3.2958823529411765|        -2.4|        9.76|
+----------------+--------------------+------------+------------+

